In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon

def calculate_centroids(gdf):
    """Calcula los centroides de los polígonos en un GeoDataFrame."""
    gdf['XCord'] = gdf.centroid.x
    gdf['YCord'] = gdf.centroid.y
    return gdf

def raw_compactness(gdf, field, unit_conversion_ratio, beta=0):
    """Calcula la compacidad de un conjunto de datos espaciales."""
    centroids = calculate_centroids(gdf)
    sum_g_weighted = 0
    sum_population = 0
    
    for _, rowO in centroids.iterrows():
        G = 0
        for _, rowD in centroids.iterrows():
            dist = unit_conversion_ratio * np.sqrt((rowD['XCord'] - rowO['XCord'])**2 + (rowD['YCord'] - rowO['YCord'])**2)
            if dist > 0:
                weight = rowD[field] / (np.exp(beta * dist) if beta else dist)
                G += weight
        sum_g_weighted += G * rowO[field]
        sum_population += rowO[field]
    
    compactness = sum_g_weighted / sum_population if sum_population else 0
    return compactness, sum_population

def generate_grid(polygon, grid_size):
    """Genera una malla de puntos sobre un polígono."""
    minx, miny, maxx, maxy = polygon.bounds
    x_coords = np.arange(minx, maxx, grid_size)
    y_coords = np.arange(miny, maxy, grid_size)
    
    points = [Point(x, y) for x in x_coords for y in y_coords if polygon.contains(Point(x, y))]
    return gpd.GeoDataFrame(geometry=points, crs=polygon.crs)

def normalize_compactness(gdf, field, reference_density, unit_conversion_ratio, grid_size):
    """Normaliza la compacidad en base a una referencia circular."""
    total_population = gdf[field].sum()
    reference_area = total_population / reference_density
    reference_radius = np.sqrt(reference_area / np.pi) / unit_conversion_ratio
    
    center = gdf.geometry.centroid.unary_union.centroid
    reference_circle = center.buffer(reference_radius)
    reference_points = generate_grid(reference_circle, grid_size)
    reference_points[field] = total_population / len(reference_points)
    
    ref_compactness, _ = raw_compactness(reference_points, field, unit_conversion_ratio)
    case_compactness, _ = raw_compactness(gdf, field, unit_conversion_ratio)
    
    return case_compactness / ref_compactness if ref_compactness else 0

In [4]:
# Ejemplo de uso
gdf = gpd.read_file(r"C:\Users\quesa\Downloads\prueba\shp.shp")

In [ ]:
compactness, total_population = raw_compactness(gdf, 'POPTOT2020', unit_conversion_ratio=1)
print("Compacidad bruta:", compactness)


In [ ]:
normalized_compactness = normalize_compactness(gdf, 'POPTOT2020', reference_density=1000, unit_conversion_ratio=1, grid_size=100)
print("Compacidad normalizada:", normalized_compactness)